In [11]:
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [12]:
!kaggle datasets download -d kvpratama/pokemon-images-dataset

 76% 54.0M/70.9M [00:00<00:00, 295MB/s]
100% 70.9M/70.9M [00:00<00:00, 299MB/s]


In [13]:
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [14]:
!unzip pokemon-images-dataset.zip -d train

Archive:  pokemon-images-dataset.zip
  inflating: train/pokemon/pokemon/1.png  
  inflating: train/pokemon/pokemon/10.png  
  inflating: train/pokemon/pokemon/100.png  
  inflating: train/pokemon/pokemon/101.png  
  inflating: train/pokemon/pokemon/102.png  
  inflating: train/pokemon/pokemon/103.png  
  inflating: train/pokemon/pokemon/104.png  
  inflating: train/pokemon/pokemon/105.png  
  inflating: train/pokemon/pokemon/106.png  
  inflating: train/pokemon/pokemon/107.png  
  inflating: train/pokemon/pokemon/108.png  
  inflating: train/pokemon/pokemon/109.png  
  inflating: train/pokemon/pokemon/11.png  
  inflating: train/pokemon/pokemon/110.png  
  inflating: train/pokemon/pokemon/111.png  
  inflating: train/pokemon/pokemon/112.png  
  inflating: train/pokemon/pokemon/113.png  
  inflating: train/pokemon/pokemon/114.png  
  inflating: train/pokemon/pokemon/115-mega.png  
  inflating: train/pokemon/pokemon/115.png  
  inflating: train/pokemon/pokemon/116.png  
  inflating: trai

In [15]:
# Imports
import tensorflow as tf
from glob import glob
from matplotlib import pyplot
import cv2
import numpy as np
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [29]:
#Constants
EPOCHS = 1000
IMAGE_HEIGHT = 64
IMAGE_WIDTH = 64
IMAGE_CHANNELS = 4
BATCH_SIZE = 128
LATENT_DIMENSION = 128
W_INIT  = tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.02)
IMAGE_PATH = glob("/content/train/pokemon/pokemon/*")


In [17]:
def load_image(path_to_image):
  image = tf.io.read_file(path_to_image)
  image = tf.io.decode_png(image)
  image = tf.image.resize(image,size=[IMAGE_HEIGHT, IMAGE_WIDTH])
  image = tf.cast(image, dtype=tf.float32)
  image = (image - 127.5)/127.5
  return image

def dataset_pipeline(path_to_image, batch_size):
  dataset =  tf.data.Dataset.from_tensor_slices(path_to_image)
  dataset = dataset.shuffle(buffer_size=1024)
  dataset = dataset.map(load_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
  dataset = dataset.batch(batch_size)
  dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
  return dataset


In [18]:
imgs_dataset =  dataset_pipeline(path_to_image=IMAGE_PATH, batch_size=BATCH_SIZE)
for x in imgs_dataset: print(x.shape)
print(len(imgs_dataset))

(128, 64, 64, 4)
(128, 64, 64, 4)
(128, 64, 64, 4)
(128, 64, 64, 4)
(128, 64, 64, 4)
(128, 64, 64, 4)
(51, 64, 64, 4)
7


In [19]:
from tensorflow.python.ops.array_ops import BatchToSpaceND
def deconv_bloc(inputs, num_filters, kernel_size, strides, bn=True):
  x = Conv2DTranspose(
      filters=num_filters,
      kernel_size=kernel_size,
      kernel_initializer = W_INIT,
      padding="same",
      strides=strides,
      use_bias=False
  )(inputs)
  if bn:
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
  return x

def conv_block(inputs, num_filters, kernel_size, padding="same", strides=2, activation=True):
  x = Conv2D(
       filters=num_filters,
      kernel_size=kernel_size,
      kernel_initializer = W_INIT,
      padding=padding,
      strides=strides 
  )(inputs)
  if activation:
    x = LeakyReLU(alpha=0.2)(x)
    x = Dropout(0.3)(x)
  return x


def build_gen(latent_dimension):
  f = [2**i for i in range(5) ][::-1]
  filters = 32
  output_strides = 16
  h_output = IMAGE_HEIGHT//output_strides
  w_output = IMAGE_WIDTH//output_strides

  noise = Input(shape=(latent_dimension,), name="gen_noise_input")
  x = Dense(f[0] * filters * h_output * w_output, use_bias=False)(noise)
  x = BatchNormalization()(x)
  x = LeakyReLU(alpha=0.2)(x)
  x = Reshape((h_output, w_output, f[0] * filters))(x)
  for i in range(1,5):
    x = deconv_bloc(
        x,
        num_filters=f[i]*filters,
        kernel_size=5,
        strides=2,
        bn=True
    )
  x = conv_block(
        x,
        activation=False,
        kernel_size=5,
        strides=1,
        num_filters=4
    )
  fake_output = Activation("tanh")(x)
  return Model(noise, fake_output, name="generator")

def build_dicriminator():
  f = [2**i for i in range(4)]
  filters = 64
  output_strides = 16
  h_output = IMAGE_HEIGHT//output_strides
  w_output = IMAGE_WIDTH//output_strides

  image_input = Input(shape=(IMAGE_WIDTH, IMAGE_HEIGHT, 4), name="images")
  x = image_input
  for i in range(0, 4):
    x = conv_block(
        x,
        kernel_size=5,
        strides=2,
        num_filters= f[i]*filters
    )
  x = Flatten()(x)
  x = Dense(1)(x)
  return Model(image_input, x, name="discriminator")
  
dicriminator_model = build_dicriminator()
generator_model = build_gen(latent_dimension=LATENT_DIMENSION)
generator_model.summary()
dicriminator_model.summary()

Model: "generator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gen_noise_input (InputLayer  [(None, 128)]            0         
 )                                                               
                                                                 
 dense_1 (Dense)             (None, 8192)              1048576   
                                                                 
 batch_normalization (BatchN  (None, 8192)             32768     
 ormalization)                                                   
                                                                 
 leaky_re_lu_4 (LeakyReLU)   (None, 8192)              0         
                                                                 
 reshape (Reshape)           (None, 4, 4, 512)         0         
                                                                 
 conv2d_transpose (Conv2DTra  (None, 8, 8, 256)        32

In [30]:
class GAN(Model):
  def __init__(self, discriminator, generator, latent_dimension, *args, **kwargs):
    super().__init__(*args, **kwargs )
    self.discriminator = discriminator
    self.generator = generator
    self.latent_dimension = latent_dimension
  def compile(self, d_optimizer, g_optimizer, loss_funtion,*args, **kwargs):
    super().compile(*args, **kwargs)
    self.d_optimizer = d_optimizer
    self.g_optimizer =g_optimizer
    self.loss_funtion = loss_funtion

  def train_step(self, real_images):
    batch_size = tf.shape(real_images)[0]
    for _ in range(2):
      random_lantent_vectors = tf.random.normal(shape=(batch_size, self.latent_dimension))
      generated_images  = self.generator(random_lantent_vectors)
      generated_labels = tf.zeros((batch_size, 1))
      with tf.GradientTape() as ftape:
        predictions = self.discriminator(generated_images)
        dl_loss = self.loss_funtion(generated_labels, predictions)
      grads = ftape.gradient(dl_loss, self.discriminator.trainable_weights)
      self.d_optimizer.apply_gradients(zip(grads, self.discriminator.trainable_weights))

      labels = tf.ones((batch_size, 1))
      with tf.GradientTape() as rtape:
        predictions = self.discriminator(real_images)
        d2_loss = self.loss_funtion(labels, predictions)
      grads = rtape.gradient(d2_loss, self.discriminator.trainable_weights)
      self.d_optimizer.apply_gradients(zip(grads, self.discriminator.trainable_weights))


    random_lantent_vectors = tf.random.normal(shape=(batch_size, self.latent_dimension))
    misleading_labels = tf.ones((batch_size, 1))
    with tf.GradientTape() as gtape:
      predictions = self.discriminator(self.generator(random_lantent_vectors))
      g_loss = self.loss_funtion(misleading_labels, predictions)
    grads = gtape.gradient(g_loss, self.generator.trainable_weights)
    self.g_optimizer.apply_gradients(zip(grads, self.generator.trainable_weights))
    return{"d1_loss":dl_loss, "d2_loss":d2_loss, "g_loss":g_loss}
  
gan = GAN(dicriminator_model, generator_model, latent_dimension=LATENT_DIMENSION)

bce_loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=True, label_smoothing=0.1)

d_optimizer = Adam(learning_rate=0.0002, beta_1=0.5)

g_optimizer = Adam(learning_rate=0.0002, beta_1=0.5)
gan.compile(d_optimizer, g_optimizer, bce_loss_fn)

images_dataset = dataset_pipeline(path_to_image=IMAGE_PATH, batch_size=BATCH_SIZE)

def save_image(examples, epoch, n):
  examples = (examples+1)/2.0
  for i in range(n*n):
    pyplot.subplot(n, n, i+1)
    pyplot.axis("off")
    pyplot.imshow(examples[i])
  filename = f"generated/fake_pokemon_epoch{epoch+1}.png"
  pyplot.savefig(filename)
  pyplot.close()
for epoch  in range(EPOCHS):
  print("********** TRAINING THE GAN **********")
  gan.fit(images_dataset, epochs=1) 
  generator_model.save("saved_model/generator.h5")
  dicriminator_model.save("saved_model/discriminator.h5")
  n_samples = 25
  noise = np.random.normal(size=(n_samples,LATENT_DIMENSION))
  examples = generator_model.predict(noise)
  save_image(examples, epoch, int(np.sqrt(n_samples)))



gan.summary()


********** TRAINING THE GAN **********
7/7 [==============================] - 6s 397ms/step - d1_loss: 0.2222 - d2_loss: 0.2194 - g_loss: 3.1039


1/1 [==============================] - 0s 17ms/step
********** TRAINING THE GAN **********
7/7 [==============================] - 3s 397ms/step - d1_loss: 0.2161 - d2_loss: 0.2141 - g_loss: 3.0151


1/1 [==============================] - 0s 18ms/step
********** TRAINING THE GAN **********
7/7 [==============================] - 3s 402ms/step - d1_loss: 0.2189 - d2_loss: 0.2198 - g_loss: 3.1025


1/1 [==============================] - 0s 20ms/step
********** TRAINING THE GAN **********
7/7 [==============================] - 3s 404ms/step - d1_loss: 0.2484 - d2_loss: 0.2465 - g_loss: 3.2449


1/1 [==============================] - 0s 16ms/step
********** TRAINING THE GAN **********
7/7 [==============================] - 3s 406ms/step - d1_loss: 0.2179 - d2_loss: 0.2123 - g_loss: 2.9532


...

1/1 [==============================] - 0s 18ms/step
********** TRAINING THE GAN **********
7/7 [==============================] - 3s 410ms/step - d1_loss: 0.2168 - d2_loss: 0.2091 - g_loss: 2.9788


1/1 [==============================] - 0s 22ms/step
********** TRAINING THE GAN **********
7/7 [==============================] - 3s 411ms/step - d1_loss: 0.2173 - d2_loss: 0.2151 - g_loss: 2.9503


1/1 [==============================] - 0s 18ms/step
********** TRAINING THE GAN **********
7/7 [==============================] - 3s 409ms/step - d1_loss: 0.2150 - d2_loss: 0.2108 - g_loss: 2.9263


1/1 [==============================] - 0s 17ms/step
********** TRAINING THE GAN **********
2/7 [=======>......................] - ETA: 2s - d1_loss: 0.2228 - d2_loss: 0.2151 - g_loss: 3.0211

KeyboardInterrupt: ignored